In [1]:
import os

os.environ['OPENAI_API_KEY']=''
os.environ['LANGCHAIN_TRACING_V2']=''
os.environ['LANGCHAIN_ENDPOINT']=''
os.environ['LANGCHAIN_API_KEY']=''
os.environ['LANGCHAIN_PROJECT']=''

In [2]:
import os
import warnings

# 경고 무시, 기본 다운로드 경로 설정
warnings.filterwarnings("ignore")
os.environ["HF_HOME"] = "./cache/"

In [4]:
### 샘플데이터

texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

In [5]:
!pip install -qU langchain_huggingface

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token="",
)

In [7]:
%%time

embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: user 16.6 ms, sys: 1 ms, total: 17.6 ms
Wall time: 429 ms


In [8]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

[HuggingFace Endpoint Embedding]
Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


In [11]:
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [12]:
len(embedded_query)

1024

### 유사도 계산

**벡터 내적을 통한 유사도 계산**
- 벡터 내적(dot product)을 사용하여 유사도를 계산합니다.

- 유사도 계산 공식:

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

#### 벡터 내적의 수학적 의미

**벡터 내적 정의**

벡터 $\mathbf{a}$와 $\mathbf{b}$의 내적은 다음과 같이 정의됩니다:
$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

**코사인 유사도와의 관계**

벡터 내적은 다음과 같은 성질을 가집니다.
$$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

여기서,
- $\|\mathbf{a}\|$와 $\|\mathbf{b}\|$는 각각 벡터 $\mathbf{a}$와 $\mathbf{b}$의 크기(노름, Euclidean norm)입니다.
- $\theta$는 두 벡터 사이의 각도입니다.
- $\cos \theta$는 두 벡터 사이의 코사인 유사도입니다.

**벡터 내적의 유사도 해석**
내적 값이 클수록 (양의 큰 값일수록),
- 두 벡터의 크기($\|\mathbf{a}\|$와 $\|\mathbf{b}\|$)가 크고,
- 두 벡터 사이의 각도($\theta$)가 작으며 ($\cos \theta$가 1에 가까움),

이는 두 벡터가 유사한 방향을 가리키고, 크기가 클수록 더 유사하다는 것을 의미합니다.

**벡터의 크기(노름) 계산**

Euclidean norm 정의
벡터 $\mathbf{a} = [a_1, a_2, \ldots, a_n]$에 대해, Euclidean norm $\|\mathbf{a}\|$는 다음과 같이 정의됩니다:
$$ \|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

In [13]:
import numpy as np

np.array(embedded_query) @ np.array(embedded_documents).T

array([0.84186319, 0.86502318, 0.86470304, 0.89564882, 0.76847344])

In [14]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

array([3, 1, 2, 0, 4])

In [15]:
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()


[0] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[1] LangChain simplifies the process of building applications with large language models

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



### Localization

In [16]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu, mpu
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [18]:
%time

embedded_documents1 = hf_embeddings.embed_documents(texts)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs


In [19]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


### Model: BGE-m3

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [22]:
%time
embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: user 27 µs, sys: 10 µs, total: 37 µs
Wall time: 49.1 µs


In [23]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


In [24]:
import numpy as np

embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
embedded_documents = hf_embeddings.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

[Query] LangChain 에 대해서 알려주세요.
[0] LangChain simplifies the process of building applications with large language models

[1] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



### FlagEmbedding

- Dense Vector: BGE-M3의 다국어, 다중 작업 능력을 기반으로 함
- Lexical weight를 활용한 sparse embedding으로 정확한 단어 매칭을 수행
- ColBERT의 multi-vector 접근법으로 문맥을 고려한 세밀한 매칭 수행

In [25]:
!pip install -qU FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.6 MB/s eta 0:00:00
  

In [26]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name, use_fp16=True)

bge_embedded = bge_embeddings.encode(
    texts,
    batch_size=12,
    max_length=8192,  # 이렇게 긴 길이가 필요하지 않은 경우 더 작은 값을 설정하여 인코딩 프로세스의 속도를 높일 수 있습니다.
)["dense_vecs"]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

imgs/long.jpg:   0%|          | 0.00/485k [00:00<?, ?B/s]

imgs/.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

...

onnx/tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/725k [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
bge_embedded.shape

(5, 1024)

In [28]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


### Lexical
'작동 방식'

각 단어에 대해 lexical weight(어휘적 가중치)를 계산합니다. 이는 TF-IDF나 BM25 같은 방법을 사용할 수 있습니다.
문서나 쿼리의 각 단어에 대해, 해당 단어의 lexical weight를 사용하여 sparse vector의 해당 차원에 값을 할당합니다.
결과적으로 문서나 쿼리는 대부분의 값이 0인 고차원 벡터로 표현됩니다.

장점

- 단어의 중요도를 직접적으로 반영할 수 있습니다.
- 특정 단어나 구문을 정확히 매칭할 수 있습니다.
- 계산이 상대적으로 빠릅니다.

In [29]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)

# use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_sparse=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [30]:
bge_encoded["lexical_weights"]

[defaultdict(int,
             {'6888': 0.15947884,
              '195269': 0.2888245,
              '4': 0.1015741,
              '54807': 0.25699174,
              '2293': 0.143452,
              '20451': 0.12553939,
              '713': 0.18402506,
              '11529': 0.17917305,
              '5': 0.11848414}),
 defaultdict(int,
             {'15786': 0.28657466,
              '62467': 0.2997092,
              '73': 0.19681463,
              '112892': 0.2142752,
              '1029': 0.17623807,
              '90': 0.15319079,
              '70': 0.017398346,
              '9433': 0.110423155,
              '111': 0.007249683,
              '33976': 0.17502293,
              '86685': 0.2180739,
              '678': 0.07308916,
              '21334': 0.107457094,
              '46876': 0.19464725,
              '115774': 0.24058667}),
 defaultdict(int,
             {'161545': 0.21388829,
              '153816': 0.19317701,
              '193751': 0.24564141,
              '135903

In [32]:
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][2]
)
# 0 <-> 0
print(lexical_scores1)
# 0 <-> 1
print(lexical_scores2)

0.3015604605898261
0.0002422393299639225


In [33]:
num_sentences = 5

# 유사도 계산 및 출력
for i in range(num_sentences):
    for j in range(num_sentences):
        lexical_score = bge_flagmodel.compute_lexical_matching_score(
            bge_encoded["lexical_weights"][i], bge_encoded["lexical_weights"][j]
        )
        print(f"{i} <==> {j}: {lexical_score:.4f}")

0 <==> 0: 0.3016
0 <==> 1: 0.0000
0 <==> 2: 0.0002
0 <==> 3: 0.0006
0 <==> 4: 0.0040
1 <==> 0: 0.0000
1 <==> 1: 0.5145
1 <==> 2: 0.1478
1 <==> 3: 0.2017
1 <==> 4: 0.0000
2 <==> 0: 0.0002
2 <==> 1: 0.1478
2 <==> 2: 0.5018
2 <==> 3: 0.1452
2 <==> 4: 0.0000
3 <==> 0: 0.0006
3 <==> 1: 0.2017
3 <==> 2: 0.1452
3 <==> 3: 0.5084
3 <==> 4: 0.0002
4 <==> 0: 0.0040
4 <==> 1: 0.0000
4 <==> 2: 0.0000
4 <==> 3: 0.0002
4 <==> 4: 0.8304


### colBERT

**장점**
- 토큰 수준의 세밀한 매칭이 가능합니다.
- 문맥을 고려한 임베딩을 생성할 수 있습니다.
- 긴 문서에 대해서도 효과적으로 작동합니다.

In [34]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)

bge_encoded = bge_flagmodel.encode(texts, return_colbert_vecs=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [35]:
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][1]
)
# 0 <-> 0
print(colbert_scores1)
# 0 <-> 1
print(colbert_scores2)

tensor(1.)
tensor(0.3748)


In [36]:
num_sentences = 5

# 유사도 계산 및 출력
for i in range(num_sentences):
    for j in range(num_sentences):
        colbert_score = bge_flagmodel.colbert_score(
            bge_encoded["colbert_vecs"][i], bge_encoded["colbert_vecs"][j]
        )
        print(f"{i} <==> {j} (ColBERT): {colbert_score:.4f}")

0 <==> 0 (ColBERT): 1.0000
0 <==> 1 (ColBERT): 0.3748
0 <==> 2 (ColBERT): 0.2793
0 <==> 3 (ColBERT): 0.3498
0 <==> 4 (ColBERT): 0.3649
1 <==> 0 (ColBERT): 0.3487
1 <==> 1 (ColBERT): 1.0000
1 <==> 2 (ColBERT): 0.5894
1 <==> 3 (ColBERT): 0.8961
1 <==> 4 (ColBERT): 0.4096
2 <==> 0 (ColBERT): 0.2190
2 <==> 1 (ColBERT): 0.5334
2 <==> 2 (ColBERT): 1.0000
2 <==> 3 (ColBERT): 0.5514
2 <==> 4 (ColBERT): 0.3236
3 <==> 0 (ColBERT): 0.2688
3 <==> 1 (ColBERT): 0.8746
3 <==> 2 (ColBERT): 0.5846
3 <==> 3 (ColBERT): 1.0000
3 <==> 4 (ColBERT): 0.3862
4 <==> 0 (ColBERT): 0.3413
4 <==> 1 (ColBERT): 0.4110
4 <==> 2 (ColBERT): 0.3499
4 <==> 3 (ColBERT): 0.3970
4 <==> 4 (ColBERT): 1.0000
